<a href="https://colab.research.google.com/github/mbagci06/lung_covid/blob/main/Unet_train_covid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# BASICS 
import os
import numpy as np
import torch

# # Colab 
# from pydrive.drive import GoogleDrive
# from google.colab import drive
# from google.colab import files
# from pydrive.auth import GoogleAuth
# from google.colab import auth
# from oauth2client.client import GoogleCredentials
# # to reach the dataset in the drive files and other functions but submittion there should not be
# drive.mount('/content/drive')
# auth.authenticate_user()
# gauth = GoogleAuth()
# gauth.credentials = GoogleCredentials.get_application_default()
# drive = GoogleDrive(gauth)
# MODEL and TRAIN
import torchvision
import torchvision.models
import torchvision.transforms as transforms
from torch.utils.data import DataLoader,Dataset
import random
from skimage import io
import torch.nn as nn
import torch.nn.functional as F
from matplotlib import pyplot as plt
from torch import optim
from PIL import Image

In [2]:
! git clone https://github.com/mbagci06/lung_covid
! pip install kaggle
! mkdir ~/.kaggle
! cp lung_covid/kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets download -d anasmohammedtahir/covidqu

Cloning into 'lung_covid'...
remote: Enumerating objects: 60, done.
remote: Counting objects: 100% (60/60), done.
remote: Compressing objects: 100% (57/57), done.
remote: Total 60 (delta 26), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (60/60), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
100% 1.15G/1.15G [00:23<00:00, 58.5MB/s]
100% 1.15G/1.15G [00:23<00:00, 52.7MB/s]


In [ ]:
!unzip "/content/covidqu.zip" -d "/content/archive/"

In [4]:
# running  .ipynb scripts to call their functions
%run lung_covid/functions.ipynb
!cp -avr lung_covid/model/ .
from model.unet import UNet, PretrainedUNet

'lung_covid/model/' -> './model'
'lung_covid/model/__init__.py' -> './model/__init__.py'
'lung_covid/model/__pycache__' -> './model/__pycache__'
'lung_covid/model/__pycache__/__init__.cpython-36.pyc' -> './model/__pycache__/__init__.cpython-36.pyc'
'lung_covid/model/__pycache__/__init__.cpython-37.pyc' -> './model/__pycache__/__init__.cpython-37.pyc'
'lung_covid/model/__pycache__/__init__.cpython-39.pyc' -> './model/__pycache__/__init__.cpython-39.pyc'
'lung_covid/model/__pycache__/unet.cpython-36.pyc' -> './model/__pycache__/unet.cpython-36.pyc'
'lung_covid/model/__pycache__/unet.cpython-37.pyc' -> './model/__pycache__/unet.cpython-37.pyc'
'lung_covid/model/__pycache__/unet.cpython-39.pyc' -> './model/__pycache__/unet.cpython-39.pyc'
'lung_covid/model/__pycache__/unet_utils.cpython-36.pyc' -> './model/__pycache__/unet_utils.cpython-36.pyc'
'lung_covid/model/__pycache__/unet_utils.cpython-37.pyc' -> './model/__pycache__/unet_utils.cpython-37.pyc'
'lung_covid/model/__pycache__/unet_util

In [6]:
# The Calling the validaton dataset
val=Lung_Dataset(split = "Val",class_folder='covid',shuffle=True,sample_size=700)# The length of the validation dataset to calculate the average value of the loss and accuracy
# The size of the dataset for calculate average datas
val_size=val.__len__()
# The Train dataset 
tra=Lung_Dataset(split = "Train",class_folder='xovid',shuffle=True,sample_size=5000)
tra_size=tra.__len__()
# # The transform functions to conver the PIL images to the tensors


# The device looks for GPU if cannot find goes with CPU
device=torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# The model is called by this function from the model.py file 
net = PretrainedUNet( in_channels=1, out_channels=2,batch_norm=True).to(device)
# The adam optimizer for optimization the values will be same for the other model training
optimizer = optim.Adam(net.parameters(), lr= 0.0001)
# The Loss function Cross entropy the value 
criterion = nn.CrossEntropyLoss()


Downloading: "https://download.pytorch.org/models/vgg11-8a719046.pth" to /root/.cache/torch/hub/checkpoints/vgg11-8a719046.pth


  0%|          | 0.00/507M [00:00<?, ?B/s]

In [7]:
batch_size=25
# loading images and masks for mini batches
data_loader = torch.utils.data.DataLoader(
        tra, batch_size=batch_size,  num_workers=2, drop_last=True)

data_loader_val = torch.utils.data.DataLoader(
        val, batch_size=batch_size,  num_workers=2, drop_last=True)

In [8]:
# The stiation of the GPU
!nvidia-smi

Tue Jun  7 20:06:32 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    33W / 250W |   1039MiB / 16280MiB |      5%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [9]:
# The number of the epoch 
epoch_number=70
# list for the calculating validation Loss
val_loss_array=[]
# list for the calculating train Loss
train_loss_array=[]
# list for the calculating validation accuracy
acc_val_array=[]
# list for the calculating train accuracy
acc_train_array=[]
for t in range(epoch_number):
    # reset the loss and the accuracy values each epoch 
    val_loss=0
    train_loss=0
    acc_val=0
    acc_train=0
    for batch_idx, sample in enumerate(data_loader):
        # Train data preparing 
        imgs , true_masks = sample['image'],sample['mask']
        imgs = imgs.to(device=device)
        # The negative takes the masks and doubles the channels with inverse (0 to 1) shape  
        true_masks=Negative_mask(true_masks,imgs.size(dim=0))
        true_masks = true_masks.to(device=device)
        net.train()
        # zero the parameter gradients
        optimizer.zero_grad()
        

        with torch.set_grad_enabled(True):
            # Train data evaluation
            masks_pred = net(imgs)
            loss = criterion(masks_pred, true_masks)
            train_loss += loss.item()
            # Updating the params
            loss.backward()
            optimizer.step()
            # Acc calc train
            acc_train+=dice( masks_pred, true_masks)
    for batch_idx, sample in enumerate(data_loader_val):
        # Validation data preparing  and calling 
        # calcel gradient feature 
        net.eval()

        # Call iamge and mask
        imgs , true_masks = sample['image'],sample['mask']

        imgs = imgs.to(device=device)
        true_masks=Negative_mask(true_masks,imgs.size(dim=0))
        true_masks = true_masks.to(device=device)

        # zero the parameter gradients
        optimizer.zero_grad()

        with torch.set_grad_enabled(False):
            # Validation evaluation 
            masks_pred = net(imgs)
            masks_pred = masks_pred.to(device=device)
            loss_val = criterion(masks_pred, true_masks)
            acc_val +=dice( masks_pred, true_masks)
            val_loss += loss_val.item()

    batch_size
    print('Epoch :',t+1,'/',str(epoch_number))
    print('Validation Loss:v',(val_loss)/(val_size/batch_size))
    print('Validation Acc:',(acc_val)/(val_size/batch_size))
    print('Train Acc:',(acc_train)/(tra_size/batch_size))
    print('Train  Loss   :t',(train_loss)/(tra_size/batch_size))
    # Store epoch progress
    val_loss_array.append((val_loss)/(val_size/batch_size))
    train_loss_array.append((train_loss)/(tra_size/batch_size))
    acc_val_array.append((acc_val)/(val_size/batch_size))
    acc_train_array.append((acc_train)/(tra_size/batch_size))

Epoch : 1 / 70
Validation Loss:v 0.26095991155930925
Validation Acc: 0.926199448960168
Train Acc: 0.7804626971183448
Train  Loss   :t 0.3233148048875669
Epoch : 2 / 70
Validation Loss:v 0.22949350305965968
Validation Acc: 0.9378039836883545
Train Acc: 0.9483106002659245
Train  Loss   :t 0.2162491477018646
Epoch : 3 / 70
Validation Loss:v 0.17849737139684813
Validation Acc: 0.9484828902142388
Train Acc: 0.9583881468604051
Train  Loss   :t 0.1721518383013652
Epoch : 4 / 70
Validation Loss:v 0.15468191940869605
Validation Acc: 0.9567359345299857
Train Acc: 0.9649094969609776
Train  Loss   :t 0.14078117649234798
Epoch : 5 / 70
Validation Loss:v 0.1345633345523051
Validation Acc: 0.9575821608304977
Train Acc: 0.9693771540766878
Train  Loss   :t 0.11857886810671106
Epoch : 6 / 70
Validation Loss:v 0.1954014758978571
Validation Acc: 0.9374662509986332
Train Acc: 0.970788571040262
Train  Loss   :t 0.10348269885781625
Epoch : 7 / 70
Validation Loss:v 0.13088589560772693
Validation Acc: 0.954823

In [10]:
import copy
# Save Model to the drive folder 
best_model_wts = copy.deepcopy(net.state_dict())
torch.save({    
    'epoch':epoch_number,
    'model_state_dict':best_model_wts,
    'optimizer_state_dict': optimizer.state_dict()
     
},os.path.join(os.getcwd(),'UNet_model_covid_pre_{}_epoch.pt'.format(epoch_number)))

In [11]:
# Save loss and acc information to the txt 
textfile = open("UNet_model_covid_pre_{}_epo.txt".format(epoch_number), "w")
# The form of the saved values are important when we used them to plot
biglist=val_loss_array+   train_loss_array+    acc_val_array+    acc_train_array
textfile. write("running_loss_array+   train_loss_array+    acc_val_array+    acc_train_array"+ "\n")
for element in [biglist]:
    textfile. write(str(element) + "\n")
textfile. close()
